In [1]:
import re
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
import random

# Dataset Scraping

## First Step:
- Write a function that creates a table of all the matches that have been played in the period under investigation
- Extract the relevant info

In [2]:
# Getting the match data for a single season


def get_matches_in_season(season, league):

    match_data = pd.DataFrame(columns=['Date', 'Home Team', 'Home XG', 'Score',
                                  'Away XG', 'Away Team', 'Attendance', 'Match Link'])

    for s in season:

        url = f"https://fbref.com/en/comps/20/{s[0]}-{s[1]}/schedule/{s[0]}-{s[1]}-{league}-Scores-and-Fixtures"
        print(url)
        response = requests.get(url)
        if response.status_code == 200:
            result = BeautifulSoup(response.content, "lxml")

            matches = result.find('tbody').find_all('tr')



            counter = len(match_data)

            for match in matches:

                try:
                    # Add code to make this date-time
                    match_data.loc[counter,'Date'] = match.find('td', {'data-stat':'date'}).get_text()

                    match_data.loc[counter,'Home Team'] = match.find('td', {'data-stat':'home_team'}).get_text()

                    match_data.loc[counter,'Home XG'] = match.find('td', {'data-stat':'home_xg'}).get_text()

                    match_data.loc[counter,'Score'] = match.find('td', {'data-stat':'score'}).get_text()

                    match_data.loc[counter,'Away XG'] = match.find('td', {'data-stat':'away_xg'}).get_text()

                    match_data.loc[counter,'Away Team'] = match.find('td', {'data-stat':'away_team'}).get_text()

                    match_data.loc[counter,'Attendance'] = match.find('td', {'data-stat':'attendance'}).get_text()

                    # The Match Link can later also function as ID
                    match_data.loc[counter,'Match Link'] = match.find('td', {'data-stat':'match_report'}).find('a').get('href')

                    #Store the outcome variable (winner)
                    outcome = list(map(int, match_data.loc[counter, 'Score'].strip().split('–')))
                    if outcome[0]>outcome[1]:
                        match_data.loc[counter,'Outcome'] = 2
                    elif outcome[1]>outcome[0]:
                         match_data.loc[counter,'Outcome'] = 0
                    else:
                        match_data.loc[counter,'Outcome'] = 1

                    counter += 1
                except:

                    continue
        else:
            print(f'Match data season {s[0]}-{s[1]} could not be loaded: Request failed')

        if league == "Bundesliga" and len(match_data)%306 != 0:

            match_data = match_data.iloc[:(len(match_data)-2),:]

    return match_data

## Second Step
- Function that for each match gets and stores all the player information
- SO basically we get the table with the player performances here

In [3]:
def get_player_per_game_info(matches):

    player_per_game = pd.DataFrame()

    for match_nr in range(len(matches)):

        print(match_nr)

        game = matches.loc[match_nr,'Match Link']

        match_page_url = f'https://fbref.com/{game}'

        if match_nr % 15 == 0:
            wait_time = random.uniform(100, 180)
            time.sleep(wait_time)

        response = requests.get(match_page_url)

        while response.status_code != 200:
            print('In Jail')
            print(len(player_per_game))
            time.sleep(3660)
            response = requests.get(match_page_url)

        result = BeautifulSoup(response.content, "lxml")

        stats_table = result.find('div',{'id':'content'}).find_all('div', {'class':'table_wrapper tabbed'})

        away_counter = 0

        for table_body in stats_table[:2]:

            table_collection = table_body.find_all('table')

            temp_list = []

            for table in table_collection:

                counter = 0

                temp_df = pd.DataFrame()

                for row in table.find('tbody').find_all('tr'):

                    temp_df.loc[counter,'Match ID'] = matches.loc[match_nr,'Match Link']

                    if away_counter:

                        temp_df.loc[counter,'Team'] = matches.loc[match_nr,'Away Team']

                    else:

                        temp_df.loc[counter,'Team'] = matches.loc[match_nr,'Home Team']

                    temp_df.loc[counter,'Date'] = matches.loc[match_nr,'Date']

                    temp_df.loc[counter,'Link'] = row.find('th').find('a').get('href')

                    for stat in row:
                        try:
                            temp_df.loc[counter,stat.get('data-stat')] = stat.get_text()
                        except:
                            temp_df.loc[counter,stat.get('data-stat')] = stat.get_text()

                    counter +=1

                temp_list.append(temp_df)

            temp_list = [df.set_index(['Link', 'Date']) for df in temp_list]

            final_table = temp_list[0]

            for tbl in temp_list[1:]:
                cols_to_use = tbl.columns.difference(final_table.columns)

                final_table = pd.merge(final_table, tbl[cols_to_use], left_index=True, right_index=True, how='outer')

            if len(player_per_game) == 0:

                player_per_game = final_table

            else:

                player_per_game = pd.concat([player_per_game,final_table],axis=0)

            away_counter += 1

    return player_per_game

## Third Step: Creating the Test-Set
- Based on the player database, we go over all the matches and take the aggreagted expected stats based on the player data before the game happened
  - So if we look at a Match of Manchester City in January 2024, we compute what would have been the expected goals etc. for that match based on the players in the line-up and by aggregating across their averages
- I also introduced the opportunity to set a window for "form", adding the stats across only the last five games

In [4]:
def create_final_set(player_per_game, match_data, window=5):
    player_per_game.iloc[:,8:] = player_per_game.iloc[:, 8:].apply(pd.to_numeric, errors='coerce')
    player_per_game['Date_col'] = player_per_game.index.get_level_values('Date')
    player_per_game["Date_col"] = pd.to_datetime(player_per_game["Date_col"])
    match_data['Date'] = pd.to_datetime(match_data['Date'])


    for matchnr in range(len(match_data)):
        print(matchnr)

        player_per_game['Link'] = player_per_game.index.get_level_values('Link')

        # Get all the players that have played in the game (all the rows of the stats too)
        players_in_game = player_per_game[player_per_game['Match ID']==match_data.loc[matchnr,'Match Link']][['Link', 'Team']]

        #
        home_team = players_in_game[players_in_game['Team']==match_data.loc[matchnr,'Home Team']]['Link']

        away_team = players_in_game[players_in_game['Team']==match_data.loc[matchnr,'Away Team']]['Link']

        home_agg_data_general = player_per_game[player_per_game['Link'].isin(home_team)]

        home_agg_data_general = home_agg_data_general.iloc[:, 8:]

        home_agg_final_set = home_agg_data_general[home_agg_data_general['Date_col']<match_data.loc[matchnr,'Date']].iloc[:,:95]
        ## Home Team
        #Find columns for Home Total
        #print(home_agg_final_set)
        agg_total_home = home_agg_final_set.groupby(level='Link').mean()

        col_means_home_tot = agg_total_home.apply(lambda x: x.sum(), axis=0)

        #Find columns for last 5 home
        agg_five_home = home_agg_final_set.groupby(level='Link').tail(window)

        agg_five_home = agg_five_home.groupby(level='Link').mean()

        col_means_home_five = agg_five_home.apply(lambda x: x.sum(), axis = 0)

        ## Away Team
        away_agg_data_general = player_per_game[player_per_game['Link'].isin(away_team)]

        away_agg_data_general = away_agg_data_general.iloc[:, 8:]

        away_agg_final_set = away_agg_data_general[away_agg_data_general['Date_col']<match_data.loc[matchnr,'Date']].iloc[:,:95]

        #Find columns for Away Total
        agg_total_away = away_agg_final_set.groupby(level='Link').mean()

        col_means_away_tot = agg_total_away.apply(lambda x: x.sum(), axis=0)

        #Find columns for last 5 away
        agg_five_away = away_agg_final_set.groupby(level='Link').tail(window)

        agg_five_away = agg_five_away.groupby(level='Link').mean()

        col_means_away_five = agg_five_away.apply(lambda x: x.sum(), axis = 0)

        #Add data to match dataset
        for header in col_means_home_tot.index:

            header_lbl = f"Home_Tot_{header}"
            match_data.loc[matchnr,header_lbl] = col_means_home_tot[header]

            header_lbl = f"Away_Tot_{header}"
            match_data.loc[matchnr,header_lbl] = col_means_away_tot[header]

            header_lbl = f"Home_Five_{header}"
            match_data.loc[matchnr,header_lbl] = col_means_home_five[header]

            header_lbl = f"Away_Five_{header}"
            match_data.loc[matchnr,header_lbl] = col_means_home_five[header]
    return match_data

In [5]:
## Running the code
seasons = [[2021, 2022], [2022, 2023], [2023, 2024]]
league = 'Bundesliga'

match_data = get_matches_in_season(seasons, league)

player_per_game = get_player_per_game_info(match_data)

match_data_final = create_final_set(player_per_game, match_data, window=5)



KeyboardInterrupt: 